# Google Scraping
---

In [1]:
import pandas as pd
import requests

from config import omdb_key

import json
from pprint import pprint

from splinter import Browser
from bs4 import BeautifulSoup

import re
import time

In [2]:
from splinter import Browser
from bs4 import BeautifulSoup

In [3]:
!which chromedriver

/usr/local/bin/chromedriver


In [4]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

### Scraping IMDB for Top 250 movies
___

In [222]:
# URL to scrape for top 250 IMDb movies

url = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'

# Read all the tables in the URL

tables = pd.read_html(url)

Raw_df = tables[0]

Raw_df.head()

,Unnamed: 0,Rank & Title,IMDb Rating,Your Rating,Unnamed: 4
0,NaN,1. The Shawshank Redemption (1994),9.2,12345678910 NOT YET RELEASED Seen,NaN
1,NaN,2. The Godfather (1972),9.1,12345678910 NOT YET RELEASED Seen,NaN
2,NaN,3. The Godfather: Part II (1974),9.0,12345678910 NOT YET RELEASED Seen,NaN
3,NaN,4. The Dark Knight (2008),9.0,12345678910 NOT YET RELEASED Seen,NaN
4,NaN,5. 12 Angry Men (1957),8.9,12345678910 NOT YET RELEASED Seen,NaN


In [223]:
Raw_df = Raw_df.drop(['Unnamed: 0','Your Rating', 'Unnamed: 4'], axis = 1)
Raw_df.head(10)

,Rank & Title,IMDb Rating
0,1. The Shawshank Redemption (1994),9.2
1,2. The Godfather (1972),9.1
2,3. The Godfather: Part II (1974),9.0
3,4. The Dark Knight (2008),9.0
4,5. 12 Angry Men (1957),8.9
5,6. Schindler's List (1993),8.9
6,7. The Lord of the Rings: The Return of the K...,8.9
7,8. Pulp Fiction (1994),8.8
8,"9. The Good, the Bad and the Ugly (1966)",8.8
9,10. The Lord of the Rings: The Fellowship of ...,8.8


### Separate 'Rank & Title' column to Rank, Title and Year columns
---

In [224]:
Imdb_movies = []

for movie in Raw_df['Rank & Title']:
    
    rank = movie.split('.')[0]
    
    title1 = re.split('[.(]', movie)[1]
    title = re.sub("  ", "", title1)
        
    year = re.split('[()]', movie)[1]
  
    Imdb_movies.append({'IMDb Rank': rank,
                        'Movie Title': title,
                        'Year Released': year
                       })
    
Imdb_df = pd.DataFrame(Imdb_movies)
    
Imdb_df['IMDb Rating'] = Raw_df['IMDb Rating'] 
    
Imdb_df.head(10)

,IMDb Rank,Movie Title,Year Released,IMDb Rating
0,1,The Shawshank Redemption,1994,9.2
1,2,The Godfather,1972,9.1
2,3,The Godfather: Part II,1974,9.0
3,4,The Dark Knight,2008,9.0
4,5,12 Angry Men,1957,8.9
5,6,Schindler's List,1993,8.9
6,7,The Lord of the Rings: The Return of the King,2003,8.9
7,8,Pulp Fiction,1994,8.8
8,9,"The Good, the Bad and the Ugly",1966,8.8
9,10,The Lord of the Rings: The Fellowship of the Ring,2001,8.8


### Building the URL to scrape Google
---

In [225]:
movies = Imdb_df['Movie Title']
movies = movies.str.replace(',', '', n = -1, case=None, regex=True)
movies = movies.str.replace("'",'', n = -1, case=None, regex=True)
movies = movies.str.replace(":",'', n = -1, case=None, regex=True)
movies = movies.str.replace(".",'', n = -1, case=None, regex=True)

In [155]:
base_url = 'https://www.google.com/search?&q='

movie_list = movies.str.lower().str.replace(' ', '+', n = -1, case=None, regex=True)

query_url=[]

for movie in movie_list:
    query_url.append(f'{base_url}{movie}+watch+movie')

In [232]:
google_query_url_df = pd.DataFrame({'IMDb Rank': Imdb_df['IMDb Rank'],
                                    'Movie Title': Imdb_df['Movie Title'],
                                    'Google Query URL' : query_url
                                   })

google_query_url_df = google_query_url_df.set_index(['IMDb Rank'])

google_query_url_df.head(10)

,Movie Title,Google Query URL
IMDb Rank,,
1,The Shawshank Redemption,https://www.google.com/search?&q=Ben-Hur+watch...
2,The Godfather,https://www.google.com/search?&q=Ben-Hur+watch...
3,The Godfather: Part II,https://www.google.com/search?&q=Ben-Hur+watch...
4,The Dark Knight,https://www.google.com/search?&q=Ben-Hur+watch...
5,12 Angry Men,https://www.google.com/search?&q=Ben-Hur+watch...
6,Schindler's List,https://www.google.com/search?&q=Ben-Hur+watch...
7,The Lord of the Rings: The Return of the King,https://www.google.com/search?&q=Ben-Hur+watch...
8,Pulp Fiction,https://www.google.com/search?&q=Ben-Hur+watch...
9,"The Good, the Bad and the Ugly",https://www.google.com/search?&q=Ben-Hur+watch...


In [21]:
google_query_url_df.to_csv('Output/Google_Query_Url.csv')

#### Sample Google scraping
---

In [152]:
#sample

sample = 'Inception'

base_url = 'https://www.google.com/search?&q='

query_url = (f'{base_url}{sample}+watch+movie')

browser.visit(query_url)

time.sleep(5)

soup = BeautifulSoup(browser.html, 'lxml')

streaming = []
title = []
price = []

results1 = soup.find_all('div', class_ = 'i3LlFf')

for result in results1:
    streaming.append(result.text)
    title.append(sample.capitalize())
    
results2 = soup.find_all('div', class_ = 'V8xno')

for result in results2:
    price.append(result.text)

Sample_Streaming_df = pd.DataFrame({'Title': title,
                                    'Streaming On' : streaming,
                                    'Price' : price
                                   })

Sample_Streaming_df['Price'] = Sample_Streaming_df['Price'].str.replace("$","", case = True, regex=True)
Sample_Streaming_df['Price'] = Sample_Streaming_df['Price'].str.replace("From ","", case = True, regex=True)

Sample_Streaming_df


,Title,Streaming On,Price
0,Inception,YouTube,3.99
1,Inception,Google Play Movies & TV,3.99
2,Inception,iTunes,3.99
3,Inception,Vudu,3.99
4,Inception,Amazon Prime Video,3.99
5,Inception,Netflix,Subscription


In [153]:
Sample_Streaming_df.to_csv('Output/Sample_Google_Scraping.csv')

## Scraping Google for few movies at a time
---

In [ ]:
Streaming = []
Title = []
Price = []

count = 0

movies = google_query_url_df['Movie Title']

query_urls = google_query_url_df['Google Query URL']

base_url = 'https://www.google.com/search?&q='

In [ ]:
for i in range(8):
    
    query_url = (f'{base_url}{movies[count]}+watch+movie')

    browser.visit(query_url)

    time.sleep(3)

    soup = BeautifulSoup(browser.html, 'lxml')

    results1 = soup.find_all('div', class_ = 'i3LlFf')

    for result in results1:
        try:
            Streaming.append(result.text)
            Title.append(movies[count].capitalize())
        except:
            Streaming.append('Nan')
            Title.append('Nan')

    results2 = soup.find_all('div', class_ = 'V8xno')

    for result in results2:
        try:
            Price.append(result.text)
        except:
            Price.append('Nan')
            
    count = count + 1

print(f'{Title}, {Streaming}, {Price}')


In [ ]:
Streaming_df = pd.DataFrame({'Title': Title,
                             'Streaming On' : Streaming,
                             'Price' : Price
                            })

Streaming_df['Price'] = Streaming_df['Price'].str.replace("$","", case = True, regex=True)
Streaming_df['Price'] = Streaming_df['Price'].str.replace("From ","", case = True, regex=True)
Streaming_df['Title'] = Streaming_df['Title'].str.title()


In [ ]:
Streaming_df.head(35)

In [218]:
# Removing the extra rows

Streaming_df_1 = Streaming_df.iloc[24:]
Streaming_df_1 = Streaming_df_1.reset_index()
Streaming_df_1 = Streaming_df_1.drop(['index'],axis = 1)
Streaming_df_1

,Title,Streaming On,Price
0,The Shawshank Redemption,YouTube,3.99
1,The Shawshank Redemption,iTunes,3.99
2,The Shawshank Redemption,Google Play Movies & TV,3.99
3,The Shawshank Redemption,Vudu,3.99
4,The Shawshank Redemption,Amazon Prime Video,3.99
...,...,...,...
794,The Thing,iTunes,3.99
795,The Thing,Google Play Movies & TV,3.99
796,The Thing,Vudu,3.99
797,The Thing,Amazon Prime Video,3.99


In [ ]:
Streaming_df_1.to_csv('Output/Google_Scraping_1.csv')

### Scraped 164 movies so far
---

### Scraping remaining movies

In [164]:
Streaming = []
Title = []
Price = []

count = 164

movies = google_query_url_df['Movie Title']

query_urls = google_query_url_df['Google Query URL']

base_url = 'https://www.google.com/search?&q='

In [240]:
for i in range(10):
    
    query_url = (f'{base_url}{movies[count]}+watch+movie')

    browser.visit(query_url)

    time.sleep(3)

    soup = BeautifulSoup(browser.html, 'lxml')

    results1 = soup.find_all('div', class_ = 'i3LlFf')

    for result in results1:
        try:
            Streaming.append(result.text)
            Title.append(movies[count].capitalize())
        except:
            Streaming.append('Nan')
            Title.append('Nan')

    results2 = soup.find_all('div', class_ = 'V8xno')

    for result in results2:
        try:
            Price.append(result.text)
        except:
            Price.append('Nan')
            
    count = count + 1

#print(f'{Title}, {Streaming}, {Price}')


In [241]:
print(f'Scraped {count} movies so far')

Scraped 234 movies so far


In [242]:
Streaming_df_2 = pd.DataFrame({'Title': Title,
                               'Streaming On' : Streaming,
                               'Price' : Price
                              })

Streaming_df_2['Price'] = Streaming_df_2['Price'].str.replace("$","", case = True, regex=True)
Streaming_df_2['Price'] = Streaming_df_2['Price'].str.replace("From ","", case = True, regex=True)
Streaming_df_2['Title'] = Streaming_df_2['Title'].str.title()

Streaming_df_2

,Title,Streaming On,Price
0,Gone With The Wind,YouTube,3.99
1,Gone With The Wind,Vudu,3.99
2,Gone With The Wind,Google Play Movies & TV,3.99
3,Gone With The Wind,iTunes,3.99
4,Gone With The Wind,Amazon Prime Video,3.99
...,...,...,...
317,It Happened One Night,Google Play Movies & TV,3.99
318,Gangs Of Wasseypur,YouTube,2.99
319,Gangs Of Wasseypur,Google Play Movies & TV,2.99
320,Gangs Of Wasseypur,iTunes,3.99


In [243]:
Streaming_df_2.to_csv('Output/Google_Scraping_2.csv')

## Concatenating the two dataframes 
---

In [244]:
Complete_Stremaing_df = pd.concat([Streaming_df_1, Streaming_df_2], ignore_index=True)
Complete_Stremaing_df.

,Title,Streaming On,Price
0,The Shawshank Redemption,YouTube,3.99
1,The Shawshank Redemption,iTunes,3.99
2,The Shawshank Redemption,Google Play Movies & TV,3.99
3,The Shawshank Redemption,Vudu,3.99
4,The Shawshank Redemption,Amazon Prime Video,3.99
5,The Shawshank Redemption,Philo,Subscription
6,The Godfather,YouTube,2.99
7,The Godfather,Google Play Movies & TV,2.99
8,The Godfather,Vudu,2.99
9,The Godfather,Amazon Prime Video,2.99


In [238]:
Complete_Stremaing_df.to_csv('Output/Complete_Google_Scraping.csv')

In [231]:
import os
os.getcwd()

'/Users/swarnaguntaka/Desktop/ETL-Project'